In [82]:
import sys
sys.path.append("../")

import pandas as pd
import optuna

from run import *

In [83]:
runner = Runner(
    Input,
    Result,
    solver_cmd="solver_cmd",
    solver_version="solver_version",
    database_csv="../log/database.csv",
    input_csv="../log/input.csv",
    log_file="../log/a.log",
    is_maximize=False,
    verbose=0,
)
df = runner.read_database_df()
df = df[df.solver_version.str.startswith("multi-start-")]
df["start_count"] = df.solver_version.str[len("multi-start-"):].astype(int)
df.input_file = df.input_file.str[len("tools/in/"):len("tools/in/0000")].astype(int)
df

,input_file,solver_version,score,duration,d,n,a.sum.mean,best_score,relative_score,start_count
140160,0,multi-start-1,964,2.9051,5,10,777225.800000,332,0.344398,1
140161,1,multi-start-1,71015,2.9097,25,36,984871.680000,50095,0.705414,1
140162,2,multi-start-1,41562,2.9082,25,15,956907.920000,38667,0.930345,1
140163,3,multi-start-1,8333,2.9108,11,42,946477.090909,6895,0.827433,1
140164,4,multi-start-1,9531,2.9052,9,37,972477.333333,8060,0.845662,1
...,...,...,...,...,...,...,...,...,...,...
146655,495,multi-start-13,10676,2.9549,28,36,818769.321429,5884,0.551143,13
146656,496,multi-start-13,1169001,2.9525,47,8,997224.851064,1163001,0.994867,13
146657,497,multi-start-13,2726,2.9525,19,29,779292.526316,1723,0.632062,13
146658,498,multi-start-13,90271,2.9529,35,15,967206.457143,75761,0.839262,13


In [84]:
input_df = pd.read_csv("../log/input.csv")
input_df.input_file = input_df.input_file.str[len("tools/in/"):len("tools/in/0000")].astype(int)
input_df = input_df.set_index("input_file")
inputs = input_df.to_dict(orient="index")

In [85]:
scores = df.groupby(["input_file", "start_count"]).relative_score.mean().to_dict()

In [ ]:
from math import pow


def f(
    n: int,
    d: int,
    a_sum_mean: float,
    mx: int,
    p1: float,
    p2: float,
    p3: float,
    p4: float,
    p5: float,
    p6: float,
    p7: float,
    p8: float,
    p9: float,
    pa: float,
):
    val = (
        pow(n, p1) * pow(d, p2) / p3
        + pow(n, p4) * p5
        + pow(d, p6) * p7
        + pow(1 - a_sum_mean, p8) * p9
        + pa
    )
    return max(1, min(mx, int(round(val))))


def objective(trial: optuna.trial.Trial) -> float:
    mx = trial.suggest_int("mx", low=3, high=13)
    p1 = trial.suggest_float("p1", low=0.0, high=3.0)
    p2 = trial.suggest_float("p2", low=0.0, high=3.0)
    p3 = trial.suggest_float("p3", low=1.0, high=300.0)
    p4 = trial.suggest_float("p4", low=0.0, high=3.0)
    p5 = trial.suggest_float("p5", low=-10.0, high=10.0)
    p6 = trial.suggest_float("p6", low=0.0, high=3.0)
    p7 = trial.suggest_float("p7", low=-10.0, high=10.0)
    p8 = trial.suggest_float("p8", low=0.0, high=3.0)
    p9 = trial.suggest_float("p9", low=-10.0, high=10.0)
    pa = trial.suggest_float("pa", low=-10.0, high=10.0)
    score_sum = 0
    for i in range(500):
        n = inputs[i]["n"]
        d = inputs[i]["d"]
        a_sum_mean = inputs[i]["a.sum.mean"] / 1e6
        val = f(n, d, a_sum_mean, mx, p1, p2, p3, p4, p5, p6, p7, p8, p9, pa)
        score_sum += scores[(i, val)]
    return score_sum / 500


study = optuna.create_study(
    direction="maximize",
    storage="sqlite:///ahc031.db",
    study_name="tune-start-count-3",
    load_if_exists=True,
)
print("start optimize")
study.enqueue_trial(
    {
        "p1": 0,
        "p2": 0,
        "p3": 30,
        "p4": 0,
        "p5": 0,
        "p6": 0,
        "p7": 0,
        "p8": 0,
        "p9": 0,
        "mx": 10,
        "pa": 0,
    }
)
study.optimize(objective, n_trials=5000, show_progress_bar=True)
study.best_trial

In [ ]:
optuna.visualization

In [88]:
study.best_params

{'mx': 7,
 'p1': 0.11708210319054835,
 'p2': 0.9269509725250654,
 'p3': 200.2802386023396,
 'p4': 0.0001068337730841043,
 'p5': 4.206987028734698,
 'p6': 0.2532608214727021,
 'p7': -3.629261670342088,
 'p8': 1.256536945225955,
 'p9': 8.813425631054976,
 'pa': 4.832322014101609}